In [0]:
%run ../../config/config

In [0]:
%run ../../config/sqlconfig

In [0]:
print(bronze_product_tbl)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType, TimestampType, DecimalType
from pyspark.sql import functions as F

In [0]:
class silverProductStreamETL:
    def __init__(self, spark):
        self.spark = spark
        self.catalog = catalog
        self.schema_name = schema_name
        self.source_table = bronze_product_tbl
        self.target_table = 'silver_product_new_cast'

        #path
        self.checkpoint_path = f"{checkpoint_path}/_silver_product_cast_checkpoint"

    def read_stream(self):
        df = (
            self.spark.readStream.table(f"{self.catalog}.{self.schema_name}.{self.source_table}")
        )   
        return df   
    def change_data_type(self, df):
        df_casted = (
            df.withColumn("product_sk", F.col("product_sk").cast(IntegerType()))
            .withColumn("product_key", F.col("product_key").cast(IntegerType()))
            .withColumn("price", F.col("price").cast(DecimalType(10, 2)))
        )
        return df_casted
    
    def clean_schema(self):
        dbutils.fs.rm(self.checkpoint_path, True)
        self.spark.sql(f"DROP TABLE IF EXISTS {self.catalog}.{self.schema_name}.{self.target_table}")

    def write_stream(self, df):
        try:
            query= (
                df.writeStream
                .option("checkpointLocation", self.checkpoint_path)
                .trigger(availableNow=True)
                .table(f"{self.catalog}.{self.schema_name}.{self.target_table}")
            )
            query.awaitTermination()
        except Exception as e:
            print(f"Error: write_stream {self.target_table}: {e}")

    def run(self):
        df = self.read_stream()
        df_casted = self.change_data_type(df)
        self.write_stream(df_casted)
        print("success full run")

In [0]:
obj = silverProductStreamETL(spark)
obj.run()

In [0]:
#obj.clean_schema()

In [0]:
%sql
SELECT *FROM silver_product_new_cast limit 2

In [0]:
%sql
-- SELECT *FROM bronze_product limit 2

In [0]:
%sql
-- drop table workspace.default.silver_product_new_cast